In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

In [2]:
learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

In [3]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)

In [4]:
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [5]:
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

In [6]:
with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [7]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [8]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_tensorboard")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

(0, 0.71563762, [array([[ 0.79269594,  0.68861032],
       [-1.20728338, -0.29517072]], dtype=float32), array([[ 1.7177    ],
       [ 0.35572484]], dtype=float32)])
(1000, 0.022333965, [array([[ 5.67364979, -6.21975279],
       [-6.14471054,  6.57215261]], dtype=float32), array([[ 8.90876007],
       [ 8.4727869 ]], dtype=float32)])
(2000, 0.0063093929, [array([[ 6.62234497, -7.19256639],
       [-7.08543205,  7.52679729]], dtype=float32), array([[ 11.22315598],
       [ 10.81134701]], dtype=float32)])
(3000, 0.0027709822, [array([[ 7.14892006, -7.73299503],
       [-7.60934496,  8.05845451]], dtype=float32), array([[ 12.75990582],
       [ 12.36296463]], dtype=float32)])
(4000, 0.0014247244, [array([[ 7.53379822, -8.12767982],
       [-7.99296522,  8.44739819]], dtype=float32), array([[ 14.01666355],
       [ 13.63034058]], dtype=float32)])
(5000, 0.00078940508, [array([[ 7.84969997, -8.45132446],
       [-8.30820274,  8.76673508]], dtype=float32), array([[ 15.14112568],
       [ 14.

In [9]:
'''
Hypothesis:  [[  6.13103184e-05]
 [  9.99936938e-01]
 [  9.99950767e-01]
 [  5.97514772e-05]]
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
Accuracy:  1.0
'''

'\nHypothesis:  [[  6.13103184e-05]\n [  9.99936938e-01]\n [  9.99950767e-01]\n [  5.97514772e-05]]\nCorrect:  [[ 0.]\n [ 1.]\n [ 1.]\n [ 0.]]\nAccuracy:  1.0\n'